In [1]:
import sys

sys.path.insert(0, "..")

In [2]:
from crewai import Agent, Task
from crewai.utilities.prompts import Prompts
from src.llm.llm_config import openrouter_llm

from src.talent_selection_flow.crews.metadata_extraction_crew.guardrails import (
    validate_cvmetadata_schema,
)
from src.talent_selection_flow.crews.metadata_extraction_crew.schemas import CVMetadata

In [3]:
import yaml

# Replace 'config.yaml' with your actual file path
agent_path = 'C:/Users/tania/Desktop/multiagent system arnau/talent_selection_flow_crewai/src/talent_selection_flow/crews/metadata_extraction_crew/config/agents.yaml'
task_path = 'C:/Users/tania/Desktop/multiagent system arnau/talent_selection_flow_crewai/src/talent_selection_flow/crews/metadata_extraction_crew/config/tasks.yaml'

with open(agent_path, 'r') as file:
    data1 = yaml.safe_load(file)
with open(task_path, 'r') as file:
    data2 = yaml.safe_load(file)

In [ ]:
# Create your agent
agent = Agent(
    config=data1["cv_metadata_extractor_agent"],
    llm=openrouter_llm,
)

# Create a sample task
task = Task(
    config=data2["extract_cv_metadata_task"],
    agent=agent,
    guardrail=validate_cvmetadata_schema,
    guardrail_max_retries=3,
    output_json=CVMetadata,
)

# Create the prompt generator
prompt_generator = Prompts(
    agent=agent,
    has_tools=len(agent.tools) > 0,
    use_system_prompt=agent.use_system_prompt
)

# Generate and inspect the actual prompt
generated_prompt = prompt_generator.task_execution()

# Print the complete system prompt that will be sent to the LLM
if "system" in generated_prompt:
    print("=== SYSTEM PROMPT ===")
    print(generated_prompt["system"])
    print("\n=== USER PROMPT ===")
    print(generated_prompt["user"])
else:
    print("=== COMPLETE PROMPT ===")
    print(generated_prompt["prompt"])

# You can also see how the task description gets formatted
print("\n=== TASK CONTEXT ===")
print(f"Task Description: {task.description}")
print(f"Expected Output: {task.expected_output}")

=== SYSTEM PROMPT ===
You are CV Metadata Extractor
. You are an expert HR analyst who extracts structured info from unstructured CVs.

Your personal goal is: Extract structured metadata from CV text.


=== USER PROMPT ===

Current Task: {input}

Provide your complete response:

=== TASK CONTEXT ===
Task Description: Extract metadata for this CV: {content} Extract this metadata: - skills: comma-separated list of required skills - industries: comma-separated relevant industries - experience_level: one of {experiencelevel_options} - country: candidate's location country in ISO code Alpha-2 - summary: 1-2 sentence overview of the role - education_level: one of {educationlevel_options} - languages: comma-separated languages spoken or "unknown"

Expected Output: Return a strict JSON object with the following structure: { "skills": "skill1, skill2, ...", "industries": "industry1, industry2, ...", "experience_level": "...", "country": "...", "summary": "...", "education_level": "...", "langua

: 

In [4]:
from crewai.hooks import before_llm_call, LLMCallHookContext

@before_llm_call
def inspect_prompt(context: LLMCallHookContext) -> None:
    for msg in context.messages:
        print(f"--- {msg['role']} ---")
        print(msg['content'])
    return None

In [5]:
from src.talent_selection_flow.crews.metadata_extraction_crew.crews import (
    CVMetadataExtractorCrew,
)
from src.talent_selection_flow.crews.metadata_extraction_crew.enums import (
    EducationLevel,
    ExperienceLevel,
)

cv_example = """
# CURRICULUM VITAE: ALEX R. DEVELOPER

## CONTACT INFORMATION
+ Email: alex.dev@email.com
+ Location: Houston, Texas
+ LinkedIn: linkedin.com/in/alexdev

---

## PROFESSIONAL SUMMARY
Highly analytical Software Engineer with 5+ years of experience in Python 
and AI orchestration. Expert in building multi-agent systems using CrewAI.

---

## EXPERIENCE
+ **Senior Backend Engineer** | TechFlow Solutions (2021 - Present)
  - Led the migration of legacy pipelines to a microservices architecture.
  - Implemented automated CV screening using LLM-based gap analysis.

+ **Junior Developer** | Startup Inc. (2018 - 2021)
  - Developed RESTful APIs using FastAPI and PostgreSQL.
  - Optimized database queries, reducing latency by 30%.

---

## TECHNICAL SKILLS
+ Languages: Python, JavaScript, SQL
+ Frameworks: CrewAI, LangChain, FastAPI, Django
+ Tools: Docker, Kubernetes, Git, OpenRouter API

---

## EDUCATION
+ B.S. in Computer Science | University of Technology
"""
metadata = (
    CVMetadataExtractorCrew(
        guardrail_max_retries=3,
        verbose=True,
        human_input=False,
    )
    .crew()
    .kickoff(
        inputs={
            "content": cv_example,
            "educationlevel_options": "/".join(EducationLevel),
            "experiencelevel_options": "/".join(ExperienceLevel),
        }
    )
)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  CV metadata extraction crew                                                                                    │
│  ID:                                                                                                            │
│  1e103ecd-b424-4737-9973-e2a31f139fae                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name: extract_metadata_task                                                                                    │
│  ID: 9e7ecd11-0a2e-4235-b772-356a6629e117                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

--- user ---
You are CV Metadata Extractor
. You are an expert HR analyst who extracts structured info from unstructured CVs.

Your personal goal is: Extract structured metadata from CV text.

Current Task: Extract metadata for this CV: 
# CURRICULUM VITAE: ALEX R. DEVELOPER

## CONTACT INFORMATION
+ Email: alex.dev@email.com
+ Location: Houston, Texas
+ LinkedIn: linkedin.com/in/alexdev

---

## PROFESSIONAL SUMMARY
Highly analytical Software Engineer with 5+ years of experience in Python 
and AI orchestration. Expert in building multi-agent systems using CrewAI.

---

## EXPERIENCE
+ **Senior Backend Engineer** | TechFlow Solutions (2021 - Present)
  - Led the migration of legacy pipelines to a microservices architecture.
  - Implemented automated CV screening using LLM-based gap analysis.

+ **Junior Developer** | Startup Inc. (2018 - 2021)
  - Developed RESTful APIs using FastAPI and PostgreSQL.
  - Optimized database queries, reducing latency by 30%.

---

## TECHNICAL SKILLS
+ La

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CV Metadata Extractor                                                                                   │
│                                                                                                                 │
│  Task: Extract metadata for this CV:                                                                            │
│  # CURRICULUM VITAE: ALEX R. DEVELOPER                                                                          │
│                                                                                                                 │
│  ## CONTACT INFORMATION                                                                                         │
│  + Email: alex.dev@email.com                                                                                    │
│  + Location: Houston, Texas                                                                                     │
│  + LinkedIn: linkedin.com/in/alexdev                                                                            │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## PROFESSIONAL SUMMARY                                                                                        │
│  Highly analytical Software Engineer with 5+ years of experience in Python                                      │
│  and AI orchestration. Expert in building multi-agent systems using CrewAI.                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## EXPERIENCE                                                                                                  │
│  + **Senior Backend Engineer** | TechFlow Solutions (2021 - Present)                                            │
│    - Led the migration of legacy pipelines to a microservices architecture.                                     │
│    - Implemented automated CV screening using LLM-based gap analysis.                                           │
│                                                                                                                 │
│  + **Junior Developer** | Startup Inc. (2018 - 2021)                                                            │
│    - Developed RESTful APIs using FastAPI and PostgreSQL.                                                       │
│    - Optimized database queries, reducing latency by 30%.                                                       │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ## TECHNICAL SKILLS                                                                                            │
│  + Languages: Python, JavaScript, SQL                                                                           │
│  + Frameworks: CrewAI, LangChain, FastAPI, Django                                                               │
│  + Tools: Docker, Kubernetes, Git, OpenRouter API      

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CV Metadata Extractor                                                                                   │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"skills": "Python, JavaScript, SQL, CrewAI, LangChain, FastAPI, Django, Docker, Kubernetes, Git, OpenRouter   │
│  API", "industries": "Software Development, AI, Backend Engineering", "experience_level": "senior", "country":  │
│  "US", "summary": "Highly analytical Software Engineer with 5+ years of experience in Python and AI             │
│  orchestration. Expert in building multi-agent systems using CrewAI.", "education_level": "bachelor",           │
│  "languages": "unknown"}                                                                                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 🛡️ Guardrail Check ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Evaluation Started                                                                                   │
│  Name:                                                                                                          │
│  def validate_cvmetadata_schema(result: TaskOutput)...                                                          │
│  Status:                                                                                                        │
│  🔄 Evaluating                                                                                                  │
│  Attempt:                                                                                                       │
│  1                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  extract_metadata_task                                                                                          │
│  Agent:                                                                                                         │
│  CV Metadata Extractor                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── 🛡️ Guardrail Success ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Passed                                                                                               │
│  Name:                                                                                                          │
│  Validation Successful                                                                                          │
│  Status:                                                                                                        │
│  ✅ Validated                                                                                                   │
│  Attempts:                                                                                                      │
│  1                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  CV metadata extraction crew                                                                                    │
│  ID:                                                                                                            │
│  1e103ecd-b424-4737-9973-e2a31f139fae                                                                           │
│  Final Output: {"skills": "Python, JavaScript, SQL, CrewAI, LangChain, FastAPI, Django, Docker, Kubernetes,     │
│  Git, OpenRouter API", "industries": "Software Development, AI, Backend Engineering", "experience_level":       │
│  "senior", "country": "US", "summary": "Highly analytical Software Engineer with 5+ years of experience in      │
│  Python and AI orchestration. Expert in building multi-agent systems using CrewAI.", "education_level":         │
│  "bachelor", "languages": "unknown"}                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯